# 03.Entrenamiento del modelo

## Importacion de la Data

In [1]:
import sqlite3
import datetime
import pandas as pd

# Guardar en un dataframe
def save_to_dataframe(db_path, table_name):
    # Conectar a la base de datos SQLite
    conn = sqlite3.connect(db_path)
    
    # Leer la tabla en un DataFrame de pandas
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    
    # Cerrar la conexión
    conn.close()
    
    return df




### Noticias

In [3]:
df_news=save_to_dataframe('../Datos/data.db', 'news')
df_news.head(10)

,fecha,fuente,titulo,descripcion,url
0,20/04/2025,Eldiario.es,Del huerto urbano a los mejores restaurantes: ...,Un proyecto para sacar de la exclusión a gente...,https://www.eldiario.es/ballenablanca/economia...
1,20/04/2025,Criptonoticias.com,Solana lleva la batuta del alza esta semana y ...,"Solana lidera con alza del 20% a USD 136,50. S...",https://www.criptonoticias.com/mercados/solana...
2,20/04/2025,Eldiario.es,El precio del plátano sigue sin techo: el cons...,"Los cosecheros locales, según registros del MA...",https://www.eldiario.es/canariasahora/eldiario...
3,20/04/2025,Forbes.com.mx,Empobrecer al prójimo es perjudicarse: arancel...,Forbes México.\n Empobrecer al prójimo es perj...,https://forbes.com.mx/empobrecer-al-projimo-es...
4,20/04/2025,Elespanol.com,Replantearse las represalias por los aranceles...,¿Y si Estados Unidos no estuviera rompiendo la...,https://www.elespanol.com/invertia/opinion/202...
5,20/04/2025,La Nacion,El traslado a precios del nuevo dólar aún no s...,Distintas consultoras detectaron movimientos s...,https://www.lanacion.com.ar/economia/el-trasla...
6,19/04/2025,Cointelegraph,El equilibrio del inversor: Rentabilidad y con...,"El inversor inteligente controla gastos, anali...",https://es.cointelegraph.com/news/the-investor...
7,19/04/2025,La Nacion,El imperio del dólar barato,Milei apela a la táctica del miedo para limita...,https://www.lanacion.com.ar/politica/el-imperi...
8,19/04/2025,La Nacion,La oposición quiere apurar un proyecto que ext...,El objetivo es evitar que vuelvan a subir en j...,https://www.lanacion.com.ar/politica/la-oposic...
9,18/04/2025,La Nacion,Maximiliano Pullaro no tuvo suerte,Las elecciones en Santa Fe dicen mucho sobre l...,https://www.lanacion.com.ar/ideas/maximiliano-...


### Historico

In [5]:
df_hist=save_to_dataframe('../Datos/data.db', 'historic')
df_hist.head(100)

,Close_COP=X,High_COP=X,Low_COP=X,Open_COP=X,Volume_COP=X,SMA_30
0,4045.770020,4050.040039,4041.000000,4045.770020,0,NaN
1,4026.000000,4032.250000,4006.030029,4029.250000,0,NaN
2,4020.239990,4043.110107,4018.939941,4020.239990,0,NaN
3,4039.750000,4040.219971,4039.000000,4039.750000,0,NaN
4,4033.689941,4058.820068,4015.165283,4033.689941,0,NaN
...,...,...,...,...,...,...
95,3674.100098,3685.260010,3642.520020,3674.100098,0,3756.004997
96,3656.250000,3666.629883,3617.528564,3656.250000,0,3756.757666
97,3668.100098,3713.729980,3647.331787,3668.100098,0,3756.796004
98,3713.260010,3713.260010,3672.768311,3713.260010,0,3757.589673
